In [1]:
from rrc_simulation.gym_wrapper.envs import cube_env, control_env
from rrc_simulation.gym_wrapper.envs.control_env import ResidualPolicyWrapper
from rrc_simulation.tasks import move_cube
from rrc_simulation.control import control_policy

import gym
import numpy as np

In [2]:
# initial_pose = move_cube.Pose(position=np.array([0,0,0.0325]), orientation=np.array([0,0,0,1]))
initial_pose = move_cube.Pose(np.array([0.02176933,0.11905757,0.0325]),
                              np.array([0,0,0.47478757,0.88010043]))
goal_pose =  move_cube.Pose(position=np.array([0,0,0.0825]), orientation=np.array([0,0,0,1]))

difficulty = 2
initializer = cube_env.FixedInitializer(
    difficulty, initial_pose, goal_pose
)
action_type = cube_env.ActionType.TORQUE_AND_POSITION

env = gym.make(
    "rrc_simulation.gym_wrapper:real_robot_challenge_phase_1-v1",
    initializer=initializer,
    action_type=action_type,
    visualization=False,
)

policy = control_policy.HierarchicalControllerPolicy(action_space=env.action_space,
            initial_pose=initial_pose, goal_pose=goal_pose,
            load_dir='./scripts/push_reorient/push_reorient_s0')
env = ResidualPolicyWrapper(env, policy)

init position: [0.02176933 0.11905757 0.0325    ], goal position: [0.     0.     0.0825], dist: 0.13095269643254315
init orientation: [0.         0.         0.47478757 0.88010043], goal orientation: [0 0 0 1]


Loading from ./scripts/push_reorient/push_reorient_s0/pyt_save/model99.pt.


loaded policy from ./scripts/push_reorient/push_reorient_s0


In [3]:
is_done = False
observation = env.reset()
accumulated_reward = 0

if isinstance(policy, (control_policy.HierarchicalControllerPolicy, 
                       control_policy.ImpedanceControllerPolicy)):
    policy.set_waypoints(env.platform, observation)

while not is_done:
    action = policy.predict(observation)
    observation, reward, is_done, info = env.step(action)
    accumulated_reward += reward


In [4]:
reward

-0.25868281864735637